<a href="https://colab.research.google.com/github/aarsanjani/meansquares/blob/master/version2/modeling/where_modeling/TX_AR_Similarity_PCA_SupetSetData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal

This colab is for cosine similarity 
* to find the counties from Iwoa, that are similar to Winsconsin widespread counties


# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"
import warnings; warnings.simplefilter('ignore')

In [ ]:
#drive.flush_and_unmount(600000)

In [ ]:
!ls '/content/drive/Shareddrives/CMPE 295- Master Project/projectdata-2021'

 All_CovidDataFeb20.csv
 alldatasets_Feb20.csv
 All_GoogleMobilityDataFeb20.csv
 All_SocialDistancingDataFeb20.csv
 AllStates_Superset_dataset_Feb20.csv
 AllStates_Superset_dataset_Feb21.csv
 CA_alldatasets_Feb05.csv
 CA_alldatasets_Jan13.csv
 CA_CombinedData_Labeled_CDCGuideline_Feb05.csv
 CA_CombinedData_Labeled_CDCGuideline_Feb06.csv
 CA_CombinedData_Labeled_CDCGuideline_Jan15.csv
 CA_combined_Feb04.csv
 CA_CovidDataJan10.csv
 CA_dataset_Feb11.csv
 CA_GoogleMobilityDataJan10.csv
 CA_MarylandData_Labeled_CDCGuideline_Feb05.csv
 CA_MarylandData_Labeled_CDCGuideline_Feb06.csv
 CA_MarylandData_Labeled_CDCGuideline_Jan15.csv
 CA_processedMarylandData_Feb04.csv
 CA_processedMarylandData_Feb05.csv
 CA_processedMarylandData_Feb12.csv
 CA_processedMarylandData_Jan11.csv
 CA_SocialDistancingDataJan10.csv
 CA_socialDistancingInertiaData_Feb04.csv
 CA_SuperData_Labeled_CDCGuideline_Feb13.csv
 CA_SuperData_Labeled_Guideline_Feb26.csv
 CA_Superset_bins_dataset_Feb20.csv
 CA_Superset_dataset_Feb12

# Import package

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler as ss
import itertools
from scipy import linalg
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
location = '/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/'
all_states_data = pd.read_csv(location+'AllStates_Superset_dataset_Mar26.csv', low_memory=False)
all_states_data.head(2)

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k
0,2020-03-01,Alabama,1001,0,Autauga County,0.0,7.194,49,15.0,13.0,0.0,0.0,3.0,0.0,35.853419,Alabama,35,22,2.81,41.6,1.0,32.9,0.26,2.55,0,55601,-1.2,0.11,19,58786,19.0,2.8,48.68,19,101,3.79,0.33,0.025,0.0,0.0,37,47.1,0.0,0.0,0.0,0,0,0.0,28,0.4,3.5,3.2,0.3,0.0,0.0,0.0
1,2020-03-02,Alabama,1001,0,Autauga County,0.0,14.587,100,15.0,4.0,0.0,0.0,5.0,-1.0,35.853419,Alabama,11,13,3.33,40.9,0.5,42.0,0.80,2.53,0,55601,-1.9,0.11,19,58786,19.0,2.8,48.68,19,101,3.79,0.33,0.025,0.0,0.0,38,47.1,0.0,0.0,0.0,0,0,0.0,28,0.4,3.5,3.2,0.3,0.0,0.0,0.0


# Compute for all states in Southern region

In [ ]:
base_state = "Texas"
other_states = ["Maryland","Virginia","West Virginia","Kentucky","North Carolina","South Carolina","Florida","Georgia",
"Tennessee","Arkansas","Oklahoma","Alabama","Mississippi","Louisiana"]

In [ ]:
location = '/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/Bayesian-Metrics/'
filename = 'All_Superset_bins_dataset_Apr05.csv'


In [ ]:
all_binning_data = pd.read_csv(location+filename, low_memory=False)
all_binning_data.head(2)

## Util method

In [ ]:
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
from tqdm import tqdm

def key_consine_similarity(tupple):
    return tupple[1]

# This method computes the distance between each vectors with predicted_vectors input
# in our case, vectors is CA data , predicted_vectors is OR data

def get_computed_similarities(vectors, predicted_vectors, reverse=False):
    data_size = len(vectors) # size of your CA data
  
    cosine_similarities = []
    simiarity_ids = []
    cosine_distances = []

    for i in range(len(predicted_vectors)): #OR
      max_cosine = 0
      max_j = -1
      for j in range(data_size): #CA
      
        cosine_sim_val = cosine_similarity(vectors[j].reshape(1,-1), predicted_vectors[i].reshape(1,-1)) 
        if max_cosine < cosine_sim_val:
            max_cosine = cosine_sim_val
            max_j = j
      cosine_similarities.append((max_j, max_cosine))
      simiarity_ids.append(max_j)
      cosine_distances.append(max_cosine)

    return simiarity_ids,cosine_distances,cosine_similarities


In [ ]:
base_state_data = all_states_data[all_states_data['Province_State'] == base_state]
base_state_data.head(3)

In [ ]:
important_cols = ['X.days..decreasing.ILI.cases',
 'Social.distancing.index',
 'PopDensity',
 'Non.work.trips.person',
 'X..Hispanic.Americans',
 'New Cases/100k population',
 'X..out.of.state.trips',
 'Median.income',
 'X..African.Americans',
 'Work.trips.person',
 'm50_index',
 'parks_percent_change_from_baseline',
 'X..Male',
 'Ventilator.shortage',
 'transit_stations_percent_change_from_baseline',
 'Testing.capacity',
 'workplaces_percent_change_from_baseline',
 'Transit.mode.share',
 'Miles.person',
 'retail_and_recreation_percent_change_from_baseline',
 'Imported.COVID.cases',
 'X..hot.spots.1000.people']


In [ ]:
base_state_data_filtered  = base_state_data[important_cols]
all_state_df = pd.DataFrame(columns = ['Date', 'Province_State', 'FIPS', 'New cases', 'County Name',
       'mask_rule_severity', 'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity', 'State_Name',
       'Social.distancing.index', 'X..staying.home', 'Trips.person',
       'X..out.of.county.trips', 'X..out.of.state.trips', 'Miles.person',
       'Work.trips.person', 'Non.work.trips.person', 'New.COVID.cases',
       'Population', 'X..change.in.consumption', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male', 'Employment.density',
       'X..hot.spots.1000.people', 'Hospital.beds.1000.people',
       'ICUs.1000.people', 'X..contact.tracing.workers.1000.people',
       'New.cases.1000.people', 'Active.cases.1000.people',
       'X.days..decreasing.COVID.cases', 'X..hospital.bed.utilization',
       'Testing.capacity', 'Tests.done.1000.people', 'X..ICU.utilization',
       'Ventilator.shortage', 'Imported.COVID.cases',
       'COVID.exposure.1000.people', 'X.days..decreasing.ILI.cases',
       'Unemployment.claims.1000.people', 'Unemployment.rate',
       'X..working.from.home', 'Cumulative.inflation.rate', 'COVID.death.rate',
       'New Cases/100k population', 'rolling_avg_new_cases/100k', 'CTNAME',
       'positivity_rate', 'similiary_score', 'cosine_distance',
       'similar_county'])


In [ ]:
for state in other_states:
  state_data = all_binning_data[all_binning_data['Province_State'] == state]
  row, cols = state_data.shape
  if row >0:
    state_data['positivity_rate'] = (state_data['New.cases.1000.people'] / state_data['Tests.done.1000.people'])* 100/1000
    state_data = state_data.replace([np.inf, -np.inf], np.nan)
    state_data_filtered = state_data[important_cols]
    state_data_filtered = state_data_filtered.fillna(0)
    simiarity_ids,cosine_distances,cosine_similarities = get_computed_similarities(vectors=base_state_data_filtered.values, predicted_vectors=state_data_filtered.values)
    state_data['similiary_score'] = cosine_similarities
    cosine_distance_array = []
    for cosine_distance in cosine_distances:
      cosine_distance_array.append(cosine_distance[0][0])
    state_data['cosine_distance'] = cosine_distance_array
    similar_county_names = []
    for id in simiarity_ids:
      similar_county_name = base_state_data.iloc[id]['County Name']
      similar_county_names.append(similar_county_name)

    print('length: ',len(similar_county_names))
    state_data['similar_county']= similar_county_names
    print(state_data.head(2))
    all_state_df = all_state_df.append(state_data)

    #break


In [ ]:
all_state_df['similar_county'].unique()

In [ ]:
all_state_df['Province_State'].unique()

## Export All states data to csv

In [ ]:

from datetime import datetime
location = '/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/cosine_distance_files/'
filename = 'TX_AllStates_PCA_SimilarityScore_'+datetime.now().strftime("%b%d")+'.csv' #%Y%m%d
print(filename)
all_state_df.to_csv(location+filename,index=False)

# Load Oregon Binning Superset Data

In [ ]:
base_state= "Texas"
similar_state= "Arkansas"

In [ ]:
or_bin_superset_data = pd.read_csv('/content/drive/Shareddrives/CMPE 295- Master Project/projectdata-2021/Arkansas_Superset_bins_dataset_Mar31.csv',low_memory=False)


In [ ]:
print(or_bin_superset_data.shape)


(1348, 58)


In [ ]:
or_bin_superset_data.head(2)

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k,State,CTNAME
0,2020-02-29,Arkansas,5021.0,0.0,Clay County,0.0,12.968571,93.714286,14.928571,0.000,0.0,0.0,1.071429,0.0,9.093629,Arkansas,16.928571,16.000,3.274286,34.257143,17.535714,39.25000,0.76000,2.512857,0.0,14847.0,-0.90000,0.0,28.0,33935.0,0.6,2.0,49.23,6.0,110.0,4.16,0.3,0.066,0.0,0.000000,43.5,43.500000,4.192857,0.003571,0.000000,0.0000,0.000,0.000714,8.5000,0.50000,4.80000,4.15000,0.2,0.00000,0.0,9.608363,AR,Clay County
1,2020-03-15,Arkansas,5021.0,0.0,Clay County,0.0,5.025000,35.937500,-13.750000,-0.125,0.0,0.0,-12.750000,0.0,9.093629,Arkansas,27.375000,18.375,2.894375,31.725000,15.868750,32.31875,0.63875,2.256250,0.0,14847.0,-11.01875,0.0,28.0,33935.0,0.6,2.0,49.23,6.0,110.0,4.16,0.3,0.066,0.0,0.051875,58.5,43.649555,15.131250,0.485625,0.420625,3.6875,0.375,0.026875,4.8125,7.73125,5.91875,20.74375,0.2,1.08625,0.0,0.000000,AR,Clay County


In [ ]:
print(or_bin_superset_data['County Name'].nunique())
or_bin_superset_data['County Name'].unique()

71


array(['Clay County', 'Cleburne County', 'Dallas County',
       'Franklin County', 'Greene County', 'Jackson County',
       'Jefferson County', 'Lawrence County', 'Lee County',
       'Madison County', 'Marion County', 'Monroe County',
       'Montgomery County', 'Perry County', 'Pike County',
       'Randolph County', 'Washington County', 'Arkansas County',
       'Ashley County', 'Baxter County', 'Benton County', 'Boone County',
       'Bradley County', 'Carroll County', 'Chicot County',
       'Clark County', 'Cleveland County', 'Columbia County',
       'Conway County', 'Craighead County', 'Crawford County',
       'Crittenden County', 'Cross County', 'Desha County', 'Drew County',
       'Faulkner County', 'Garland County', 'Grant County',
       'Hempstead County', 'Hot Spring County', 'Howard County',
       'Independence County', 'Izard County', 'Johnson County',
       'Lafayette County', 'Lincoln County', 'Little River County',
       'Logan County', 'Lonoke County', 'Mille

In [ ]:
or_bin_superset_data['positivity_rate'] = (or_bin_superset_data['New.cases.1000.people'] / or_bin_superset_data['Tests.done.1000.people'])* 100/1000
or_bin_superset_data.head(2)

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k,State,CTNAME,positivity_rate
0,2020-02-29,Arkansas,5021.0,0.0,Clay County,0.0,12.968571,93.714286,14.928571,0.000,0.0,0.0,1.071429,0.0,9.093629,Arkansas,16.928571,16.000,3.274286,34.257143,17.535714,39.25000,0.76000,2.512857,0.0,14847.0,-0.90000,0.0,28.0,33935.0,0.6,2.0,49.23,6.0,110.0,4.16,0.3,0.066,0.0,0.000000,43.5,43.500000,4.192857,0.003571,0.000000,0.0000,0.000,0.000714,8.5000,0.50000,4.80000,4.15000,0.2,0.00000,0.0,9.608363,AR,Clay County,0.0
1,2020-03-15,Arkansas,5021.0,0.0,Clay County,0.0,5.025000,35.937500,-13.750000,-0.125,0.0,0.0,-12.750000,0.0,9.093629,Arkansas,27.375000,18.375,2.894375,31.725000,15.868750,32.31875,0.63875,2.256250,0.0,14847.0,-11.01875,0.0,28.0,33935.0,0.6,2.0,49.23,6.0,110.0,4.16,0.3,0.066,0.0,0.051875,58.5,43.649555,15.131250,0.485625,0.420625,3.6875,0.375,0.026875,4.8125,7.73125,5.91875,20.74375,0.2,1.08625,0.0,0.000000,AR,Clay County,0.0


#  Load CA Superset Labeled Data

In [ ]:
ca_labeled_superset_data = all_states_data[all_states_data['Province_State'] == base_state]
ca_labeled_superset_data.head(3)

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k
6111,2020-03-01,Texas,48057,0,Calhoun County,0.0,10.531,97,12.0,20.0,0.0,0.0,3.0,0.0,16.612237,Texas,33,22,3.01,32.5,0.1,45.1,0.33,2.69,0,21561,-10.5,0.48,22,55469,2.4,48.4,50.73,24,115,2.82,0.3,0.04,0.0,0.0,37,47.7,0.0,0.0,0.0,0,0,0.0,56,0.5,4.7,4.6,0.2,0.0,0.0,43.290043
6112,2020-03-02,Texas,48057,0,Calhoun County,0.0,9.906,91,18.0,14.0,0.0,0.0,12.0,0.0,16.612237,Texas,11,14,3.73,34.3,0.2,36.1,0.53,3.20,0,21561,6.5,0.48,22,55469,2.4,48.4,50.73,24,115,2.82,0.3,0.04,0.0,0.0,38,47.7,0.0,0.0,0.0,0,0,0.0,56,0.5,4.7,4.6,0.2,0.0,0.0,34.435262
6113,2020-03-03,Texas,48057,0,Calhoun County,0.0,10.828,100,15.0,12.0,0.0,0.0,8.0,0.0,16.612237,Texas,12,14,3.90,33.5,0.0,34.1,0.63,3.27,0,21561,8.8,0.48,22,55469,2.4,48.4,50.73,24,115,2.82,0.3,0.04,0.0,0.0,39,47.7,0.0,0.0,0.0,0,0,0.0,63,0.5,4.7,4.6,0.2,0.0,0.0,27.548209


In [ ]:
ca_labeled_superset_data['Province_State'].unique()

array(['Texas'], dtype=object)

In [ ]:
ca_labeled_superset_data.shape

(51965, 56)

In [ ]:
#ca_labeled_superset_data = ca_labeled_superset_data[ca_labeled_superset_data['growth_label']=='widespread']

In [ ]:
ca_labeled_superset_data.shape

(51965, 56)

In [ ]:
print("CA labeled:", ca_labeled_superset_data.Date.min(),ca_labeled_superset_data.Date.max())
print("Oregon Bining:", or_bin_superset_data.Date.min(),or_bin_superset_data.Date.max())

CA labeled: 2020-03-01 2020-12-28
Oregon Bining: 2020-02-29 2020-12-15


# Filtering widespread data from CA

In [ ]:
#ca_widespread_data = ca_labeled_superset_data[ca_labeled_superset_data['growth_label']=='widespread']
#print(ca_widespread_data.shape)
#ca_widespread_data.head(2)

# Using PCA Columns

## Column Mapping

In [ ]:
ca_labeled_superset_data.columns

Index(['Date', 'Province_State', 'FIPS', 'New cases', 'County Name',
       'mask_rule_severity', 'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity', 'State_Name',
       'Social.distancing.index', 'X..staying.home', 'Trips.person',
       'X..out.of.county.trips', 'X..out.of.state.trips', 'Miles.person',
       'Work.trips.person', 'Non.work.trips.person', 'New.COVID.cases',
       'Population', 'X..change.in.consumption', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male', 'Employment.density',
       'X..hot.spots.1000.people', 'Hospital.beds.1000.people',
       'ICUs.1000.people', 'X..contact.tr

'Province_State', 'FIPS', 'County Name','PopDensity', 'State_Name','State', 'CTFIPS', 'CTNAME', 'STFIPS',
'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male', 'Employment.density','Testing.capacity', 'Ventilator.shortage','Unemployment.rate','Unemployment.claims.1000.people',

In [ ]:
or_bin_superset_data.columns

Index(['Date', 'Province_State', 'FIPS', 'New cases', 'County Name',
       'mask_rule_severity', 'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity', 'State_Name',
       'Social.distancing.index', 'X..staying.home', 'Trips.person',
       'X..out.of.county.trips', 'X..out.of.state.trips', 'Miles.person',
       'Work.trips.person', 'Non.work.trips.person', 'New.COVID.cases',
       'Population', 'X..change.in.consumption', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male', 'Employment.density',
       'X..hot.spots.1000.people', 'Hospital.beds.1000.people',
       'ICUs.1000.people', 'X..contact.tr

In [ ]:
required_cols = [ 'mask_rule_severity', 'm50_index', #'FIPS',
        'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity',
        'Social.distancing.index','X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person',#'Population',
       'X..change.in.consumption', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male','Employment.density',
       'X..hot.spots.1000.people', 'Hospital.beds.1000.people',
       'ICUs.1000.people','X.days..decreasing.COVID.cases', 'X..hospital.bed.utilization',
       'Testing.capacity', 'Tests.done.1000.people', 'X..ICU.utilization',
       'Ventilator.shortage', 'Imported.COVID.cases',
       'COVID.exposure.1000.people', 'X.days..decreasing.ILI.cases',
       'Unemployment.claims.1000.people', 'Unemployment.rate',
       'X..working.from.home', 'Cumulative.inflation.rate', 'COVID.death.rate',
       'rolling_avg_new_cases/100k']

### Important columns from PCA

In [ ]:
important_cols = ['X.days..decreasing.ILI.cases',
 'Social.distancing.index',
 'PopDensity',
 'Non.work.trips.person',
 'X..Hispanic.Americans',
 'New Cases/100k population',
 'X..out.of.state.trips',
 'Median.income',
 'X..African.Americans',
 'Work.trips.person',
 'm50_index',
 'parks_percent_change_from_baseline',
 'X..Male',
 'Ventilator.shortage',
 'transit_stations_percent_change_from_baseline',
 'Testing.capacity',
 'workplaces_percent_change_from_baseline',
 'Transit.mode.share',
 'Miles.person',
 'retail_and_recreation_percent_change_from_baseline',
 'Imported.COVID.cases',
 'X..hot.spots.1000.people']


In [ ]:
if len(important_cols) > len(set(important_cols)):
   print('not unique')

len(important_cols)

22

**Summary**: The latent features (loc2vec features ) describe the probalistic causal behaviour of the pandemic. The features are responsible for describing the sudden increase or surge of the pandemic. The counties with closest 

* Surge in LA -> Surge in WA county in OR


In [ ]:
ca_labeled_superset_data_filtered = ca_labeled_superset_data[required_cols]


In [ ]:
or_bin_superset_data_filtered = or_bin_superset_data[required_cols]

#### Updated to select PCA important columns 

In [ ]:

ca_labeled_superset_data_filtered = ca_labeled_superset_data[important_cols]
or_bin_superset_data_filtered = or_bin_superset_data[important_cols]

In [ ]:
print(ca_labeled_superset_data_filtered.shape)
print(or_bin_superset_data_filtered.shape)

(51965, 22)
(1348, 22)


# KNN Similarity between CA data and OR binning reduced data



In [ ]:
#https://towardsdatascience.com/cosine-similarity-how-does-it-measure-the-similarity-maths-behind-and-usage-in-python-50ad30aad7db
#### Reference Code 
# using sklearn to calculate cosine similarity
#from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
#cos_sim=cosine_similarity(A.reshape(1,-1),B.reshape(1,-1))
#print (f"Cosine Similarity between A and B:{cos_sim}")- high is good
#print (f"Cosine Distance between A and B:{1-cos_sim}")- distance less is good

In [ ]:
or_bin_superset_data_filtered.describe().T

,count,mean,std,min,25%,50%,75%,max
X.days..decreasing.ILI.cases,1348.0,52.443650,44.460418,0.000000,18.062500,35.000000,79.100000,155.312500
Social.distancing.index,1348.0,23.182178,6.729412,8.300000,18.500000,22.345238,26.839286,49.600000
PopDensity,1348.0,23.103648,31.438424,4.299626,8.569000,11.052416,27.001253,200.176589
Non.work.trips.person,1348.0,2.765827,0.346080,1.700000,2.532885,2.757303,2.972682,4.050000
X..Hispanic.Americans,1348.0,5.507864,5.503973,0.200000,2.375000,3.300000,5.900000,33.200000
New Cases/100k population,1348.0,24.829733,31.948654,-13.923698,1.630529,14.410184,38.476998,385.563775
X..out.of.state.trips,1348.0,6.034293,7.263136,0.090909,1.534598,3.366667,7.675893,49.330000
Median.income,1348.0,41323.939169,6462.650103,28367.000000,36897.000000,41196.000000,43826.000000,64141.000000
X..African.Americans,1348.0,15.012315,16.958112,0.100000,1.300000,6.900000,28.800000,62.600000
Work.trips.person,1348.0,0.482037,0.125226,0.140000,0.404500,0.480857,0.560000,0.855714


In [ ]:
ca_labeled_superset_data_filtered.describe().T

,count,mean,std,min,25%,50%,75%,max
X.days..decreasing.ILI.cases,51965.0,130.315308,81.967869,0.000000,63.000000,119.000000,203.000000,273.000000
Social.distancing.index,51965.0,28.432964,11.415898,3.000000,20.000000,26.000000,35.000000,80.000000
PopDensity,51965.0,62.780600,154.371169,0.058884,7.118089,14.786392,37.933248,1145.193915
Non.work.trips.person,51965.0,2.827862,0.488552,0.000000,2.510000,2.800000,3.100000,11.470000
X..Hispanic.Americans,51965.0,33.950528,23.514160,3.500000,17.900000,24.400000,46.500000,99.100000
New Cases/100k population,51965.0,19.809058,64.532255,-1839.571456,0.000000,0.931246,18.121851,3295.150628
X..out.of.state.trips,51965.0,2.249789,4.993361,0.000000,0.200000,0.500000,1.900000,94.900000
Median.income,51965.0,52618.114827,12194.138884,27378.000000,44975.000000,50882.000000,57646.000000,95561.000000
X..African.Americans,51965.0,7.434685,6.851167,0.000000,1.800000,5.700000,10.800000,33.500000
Work.trips.person,51965.0,0.421551,0.150593,0.000000,0.310000,0.420000,0.510000,1.400000


In [ ]:
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
from tqdm import tqdm

def key_consine_similarity(tupple):
    return tupple[1]

# This method computes the distance between each vectors with predicted_vectors input
# in our case, vectors is CA data , predicted_vectors is OR data

def get_computed_similarities(vectors, predicted_vectors, reverse=False):
    data_size = len(vectors) # size of your CA data
    #print("len of OR:", len(predicted_vectors)) #size of OR
    #print("len of CA:", data_size)
    cosine_similarities = []
    simiarity_ids = []
    cosine_distances = []

    for i in tqdm(range(len(predicted_vectors))): #OR
      max_cosine = 0
      max_j = -1
      #print('actual : ',predicted_vectors[i],' reshaped : ',predicted_vectors[i].reshape(1,-1))
      for j in range(data_size): #CA
        ##cosine_sim_val = (1 - cosine(vectors[j], predicted_vectors[i]))
        # if i == 31:
        #   print(vectors[j].reshape(1,-1))
        #   print(predicted_vectors[i].reshape(1,-1))
        cosine_sim_val = cosine_similarity(vectors[j].reshape(1,-1), predicted_vectors[i].reshape(1,-1)) 
        if max_cosine < cosine_sim_val:
            max_cosine = cosine_sim_val
            max_j = j
      cosine_similarities.append((max_j, max_cosine))
      simiarity_ids.append(max_j)
      cosine_distances.append(max_cosine)
        #if cosine_sim_val > 0.5: # update threshold
          #continue
      #if i > 50: break
        
    #print('len cosine_similarities : ',len(cosine_similarities))
    #return sorted(cosine_similarities, key=key_consine_similarity, reverse=reverse)
    return simiarity_ids,cosine_distances,cosine_similarities

def display_top_n(sorted_cosine_similarities, n=10):
    for i in range(n):
        index, consine_sim_val = sorted_cosine_similarities[i]
        print('News Title: ', merged_data.iloc[index, 0])  
        print('News statement: ', merged_data.iloc[index, 1])  
        print('News label: ', merged_data.iloc[index, 4])  

        print('Cosine Sim Val :', consine_sim_val)
        print('---------------------------------')

In [ ]:
or_bin_superset_data_filtered.values[31].reshape(1,-1)

array([[ 0.00000000e+00,  2.37500000e+01,  1.75920795e+01,
         2.76250000e+00,  2.40000000e+00,  2.10294412e+01,
         1.17500000e+00,  4.33910000e+04,  6.00000000e-01,
         3.82500000e-01,  9.55000000e+01,  0.00000000e+00,
         4.94300000e+01,  4.80000000e+01,  0.00000000e+00,
         8.57500000e+00, -1.17500000e+01,  0.00000000e+00,
         3.88500000e+01,  1.50000000e+00,  1.33750000e+02,
         1.39000000e+02]])

In [ ]:
%%time
simiarity_ids,cosine_distances,cosine_similarities = get_computed_similarities(vectors=ca_labeled_superset_data_filtered.values, predicted_vectors=or_bin_superset_data_filtered.values)

100%|██████████| 1348/1348 [3:53:29<00:00, 10.39s/it]

CPU times: user 3h 55min 15s, sys: 3min 7s, total: 3h 58min 23s
Wall time: 3h 53min 29s


In [ ]:
print(cosine_similarities[:10][0][1])
float(cosine_similarities[:10][0][1])

[[0.99999951]]


0.9999995093689159

In [ ]:
or_bin_superset_data['FIPS'].unique()

array([5021., 5023., 5039., 5047., 5055., 5067., 5069., 5075., 5077.,
       5087., 5089., 5095., 5097., 5105., 5109., 5121., 5143., 5001.,
       5003., 5005., 5007., 5009., 5011., 5015., 5017., 5019., 5025.,
       5027., 5029., 5031., 5033., 5035., 5037., 5041., 5043., 5045.,
       5051., 5053., 5057., 5059., 5061., 5063., 5065., 5071., 5073.,
       5079., 5081., 5083., 5085., 5091., 5093., 5099., 5103., 5107.,
       5111., 5113., 5115., 5117., 5119., 5125., 5127., 5129., 5131.,
       5133., 5135., 5137., 5139., 5141., 5145., 5147., 5149.])

In [ ]:
or_bin_superset_data['similiary_score'] = cosine_similarities


# Extracting cosine distance value

In [ ]:
cosine_distance_array = []
for cosine_distance in cosine_distances:
  #print(cosine_distance[0][0])
  cosine_distance_array.append(cosine_distance[0][0])
  


or_bin_superset_data['cosine_distance'] = cosine_distance_array


# Assigning Similar counties from similarity Id

In [ ]:
#print("CA row number=",or_bin_superset_data.iloc[0]['similiary_score'][0])
similar_county_names = []
for id in simiarity_ids:
  similar_county_name = ca_labeled_superset_data.iloc[id]['County Name']
  similar_county_names.append(similar_county_name)

print('length: ',len(similar_county_names))
or_bin_superset_data['similar_county']= similar_county_names
or_bin_superset_data.head(2)

length:  1348


,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k,State,CTNAME,positivity_rate,similiary_score,cosine_distance,similar_county
0,2020-02-29,Arkansas,5021.0,0.0,Clay County,0.0,12.968571,93.714286,14.928571,0.000,0.0,0.0,1.071429,0.0,9.093629,Arkansas,16.928571,16.000,3.274286,34.257143,17.535714,39.25000,0.76000,2.512857,0.0,14847.0,-0.90000,0.0,28.0,33935.0,0.6,2.0,49.23,6.0,110.0,4.16,0.3,0.066,0.0,0.000000,43.5,43.500000,4.192857,0.003571,0.000000,0.0000,0.000,0.000714,8.5000,0.50000,4.80000,4.15000,0.2,0.00000,0.0,9.608363,AR,Clay County,0.0,"(18414, [[0.9999995093689159]])",1.0,Hardeman County
1,2020-03-15,Arkansas,5021.0,0.0,Clay County,0.0,5.025000,35.937500,-13.750000,-0.125,0.0,0.0,-12.750000,0.0,9.093629,Arkansas,27.375000,18.375,2.894375,31.725000,15.868750,32.31875,0.63875,2.256250,0.0,14847.0,-11.01875,0.0,28.0,33935.0,0.6,2.0,49.23,6.0,110.0,4.16,0.3,0.066,0.0,0.051875,58.5,43.649555,15.131250,0.485625,0.420625,3.6875,0.375,0.026875,4.8125,7.73125,5.91875,20.74375,0.2,1.08625,0.0,0.000000,AR,Clay County,0.0,"(4587, [[0.9999995803948353]])",1.0,Shelby County


In [ ]:
or_bin_superset_data.to_csv("AR_testdata_labeled.csv",index=False)

In [ ]:
or_bin_superset_data.tail()

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k,State,CTNAME,positivity_rate,similiary_score,cosine_distance,similar_county
1343,2020-10-15,Arkansas,5149.0,3.437500,Yell County,0.0,10.347250,80.000000,0.0,0.0,0.0,0.0,-15.125000,0.0,8.954998,Arkansas,13.000000,13.125000,3.777500,35.212500,0.668750,51.468750,0.495625,3.281250,3.500000,21535.0,29.550000,0.1,23.0,42361.0,2.0,20.0,49.98,11.0,85.0,4.16,0.3,0.066,0.157694,8.269375,2.312500,49.604581,8.193750,418.276250,26.197500,69.125000,199.562500,22.624375,41.562500,1.200000,6.281250,27.837500,0.690000,2.603125,15.962387,14.552721,AR,Yell County,0.000038,"(22674, [[0.9999979090794884]])",0.999998,Brazoria County
1344,2020-10-31,Arkansas,5149.0,9.466667,Yell County,0.0,9.970400,77.200000,0.0,0.0,0.0,0.0,-14.333333,0.0,8.954998,Arkansas,16.533333,16.466667,3.730000,35.113333,0.746667,50.426667,0.478667,3.253333,9.466667,21535.0,28.453333,0.1,23.0,42361.0,2.0,20.0,49.98,11.0,85.0,4.16,0.3,0.066,0.404507,9.990667,0.000000,50.490330,8.513333,469.583333,28.468667,80.466667,245.866667,27.259333,35.933333,1.060000,6.206667,27.120000,0.503333,2.526667,43.959446,32.947472,AR,Yell County,0.000086,"(12665, [[0.9999975735870775]])",0.999998,Ellis County
1345,2020-11-15,Arkansas,5149.0,10.533333,Yell County,0.0,9.646800,74.733333,0.0,0.0,0.0,0.0,-25.666667,0.0,8.954998,Arkansas,21.600000,18.866667,3.214667,35.926667,0.960000,52.313333,0.466000,2.749333,10.533333,21535.0,8.546667,0.1,23.0,42361.0,2.0,20.0,49.98,11.0,85.0,4.16,0.3,0.066,0.506660,13.302667,1.133333,50.459391,9.100000,528.228000,40.532000,79.266667,286.266667,33.782667,8.866667,1.146667,6.573333,28.733333,0.490000,2.306667,48.912623,53.512002,AR,Yell County,0.000096,"(26169, [[0.9999968473575197]])",0.999997,Comal County
1346,2020-11-30,Arkansas,5149.0,19.090909,Yell County,0.0,10.206273,79.090909,0.0,0.0,0.0,0.0,-14.909091,0.0,8.954998,Arkansas,14.909091,15.727273,3.779091,33.354545,0.790909,53.345455,0.506364,3.271818,19.090909,21535.0,29.181818,0.1,23.0,42361.0,2.0,20.0,49.98,11.0,85.0,4.16,0.3,0.066,0.817564,16.299091,2.363636,53.593823,9.636364,579.554545,41.844545,117.636364,365.454545,42.956364,22.272727,1.100000,7.036364,30.236364,0.308182,2.054545,88.650611,76.830530,AR,Yell County,0.000141,"(32387, [[0.9999957545538575]])",0.999996,Hays County
1347,2020-12-15,Arkansas,5149.0,21.100000,Yell County,0.0,9.578300,74.100000,-8.6,0.0,0.0,0.0,-31.100000,0.0,8.954998,Arkansas,21.400000,17.900000,3.055000,40.860000,0.950000,50.750000,0.423000,2.632000,21.100000,21535.0,3.920000,0.1,23.0,42361.0,2.0,20.0,49.98,11.0,85.0,4.16,0.3,0.066,1.123760,19.863000,1.000000,57.275106,10.570000,655.958000,38.993000,159.900000,470.100000,53.456000,35.000000,1.100000,7.580000,27.600000,0.290000,2.086000,97.980033,103.685031,AR,Yell County,0.000171,"(12679, [[0.9999962687069673]])",0.99

In [ ]:
or_bin_superset_data['County Name'].unique()

array(['Clay County', 'Cleburne County', 'Dallas County',
       'Franklin County', 'Greene County', 'Jackson County',
       'Jefferson County', 'Lawrence County', 'Lee County',
       'Madison County', 'Marion County', 'Monroe County',
       'Montgomery County', 'Perry County', 'Pike County',
       'Randolph County', 'Washington County', 'Arkansas County',
       'Ashley County', 'Baxter County', 'Benton County', 'Boone County',
       'Bradley County', 'Carroll County', 'Chicot County',
       'Clark County', 'Cleveland County', 'Columbia County',
       'Conway County', 'Craighead County', 'Crawford County',
       'Crittenden County', 'Cross County', 'Desha County', 'Drew County',
       'Faulkner County', 'Garland County', 'Grant County',
       'Hempstead County', 'Hot Spring County', 'Howard County',
       'Independence County', 'Izard County', 'Johnson County',
       'Lafayette County', 'Lincoln County', 'Little River County',
       'Logan County', 'Lonoke County', 'Mille

In [ ]:
print(ca_labeled_superset_data_filtered.shape)
ca_labeled_superset_data_filtered.head(2)

In [ ]:
or_bin_superset_data.head(4)

### Save Temp File to CSV [For Analysis]

In [ ]:
from datetime import datetime
location = '/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/'
filename = 'AR_SuperData_With_SimilarityScore_'+datetime.now().strftime("%b%d")+'.csv' #%Y%m%d
print(filename)
or_bin_superset_data.to_csv(location+filename,index=False)

AR_SuperData_With_SimilarityScore_Apr01.csv
